In [40]:
import weaviate
from weaviate.classes.init import Auth
import weaviate.classes.config as wc
from weaviate.classes.config import Configure
from weaviate.classes.config import ReferenceProperty
from unstructured.partition.auto import partition
from weaviate.util import generate_uuid5
import re
import requests
from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
import os
from typing import List, Dict,Tuple
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_weaviate.vectorstores import WeaviateVectorStore
from weaviate.classes import query as wvc
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from weaviate import Client
import openai
from dotenv import load_dotenv
import logging
from flair.data import Sentence
from flair.models import SequenceTagger
from fpdf import FPDF
from bs4 import BeautifulSoup
from llama_index.readers.web import BeautifulSoupWebReader
from io import BytesIO
import tempfile
from langchain.document_loaders import WebBaseLoader


In [60]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\dhanu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [5]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
openai.api_type = "azure"
openai.api_version = "2023-05-15"
openai.api_base = "h"
openai.api_key = "02jG4ocs"

weaviate_url = "hd"
weaviate_key = "tdc"

In [ ]:
azure_key = "b9a77"

In [8]:

headers = {
    "X-Azure-Api-Key": azure_key,
}


In [ ]:


# Connect to your WCD instance
client = weaviate.connect_to_wcs(
    cluster_url=weaviate_url,
    auth_credentials=weaviate.auth.AuthApiKey(weaviate_key),
    headers={
        "X-Azure-Api-Key": "02j8yABACOG4ocs"  # Replace with your OpenAI key
    }
)

client.is_ready()

INFO:httpx:HTTP Request: GET https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/.well-known/ready "HTTP/1.1 200 OK"


True

In [10]:

tagger = SequenceTagger.load('flair/ner-english')


2025-01-22 16:13:30,145 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [62]:
client.collections.delete("Documents")

INFO:httpx:HTTP Request: DELETE https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/schema/Documents "HTTP/1.1 200 OK"


In [65]:
import weaviate.classes as wvc


collection_name = "Documents"

if client.collections.exists(collection_name):  # In case we've created this collection before
    client.collections.delete(collection_name)  # THIS WILL DELETE ALL DATA IN THE COLLECTION

chunk_client = client.collections.create(
    name=collection_name,
    properties=[
        wvc.config.Property(
            name="content",
            data_type=wvc.config.DataType.TEXT
        ),
        wvc.config.Property(
            name="keyword",
            data_type=wvc.config.DataType.TEXT
        ),
        wvc.config.Property(
            name="embedding",
            data_type=wvc.config.DataType.INT
        ),
      
    ],
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),  # Use `text2vec-openai` as the vectorizer
    generative_config=wvc.config.Configure.Generative.openai(),  # Use `generative-openai` with default parameters
)

INFO:httpx:HTTP Request: GET https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/schema/Documents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/schema/Documents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/schema "HTTP/1.1 200 OK"


In [66]:
response = client.get_meta()
print(response)

INFO:httpx:HTTP Request: GET https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/meta "HTTP/1.1 200 OK"


{'grpcMaxMessageSize': 10485760, 'hostname': 'http://[::]:8080', 'modules': {'backup-gcs': {'bucketName': 'weaviate-wcs-prod-cust-us-east1-workloads-backups', 'rootName': '99285617-6cc7-4333-9eed-8a4232634556'}, 'generative-anthropic': {'documentationHref': 'https://docs.anthropic.com/en/api/getting-started', 'name': 'Generative Search - Anthropic'}, 'generative-anyscale': {'documentationHref': 'https://docs.anyscale.com/endpoints/overview', 'name': 'Generative Search - Anyscale'}, 'generative-aws': {'documentationHref': 'https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html', 'name': 'Generative Search - AWS'}, 'generative-cohere': {'documentationHref': 'https://docs.cohere.com/reference/chat', 'name': 'Generative Search - Cohere'}, 'generative-databricks': {'documentationHref': 'https://docs.databricks.com/en/machine-learning/foundation-models/api-reference.html#completion-task', 'name': 'Generative Search - Databricks'}, 'generative-friendliai': {'documentationHref': 

In [48]:
from docx import Document
from PyPDF2 import PdfReader

In [46]:
def extract_keywords(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    # Extract only entities labeled as NER
    keywords = [entity.text for entity in sentence.get_spans('ner')]
    keywords = list(set(keywords))
    return keywords

In [47]:

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = text.strip()
    return text

def fetch_content_url(url, crawled_urls=set()):
    if url in crawled_urls:
        return ""
    try:
        documents = BeautifulSoupWebReader().load_data([url])
    except Exception as e:
        logger.error(f"Failed to load content from {url}: {e}")
        return ""

    crawled_urls.add(url)

    if documents:
        doc = documents[0]
        cleaned_text = clean_text(doc.text.strip())

        soup = BeautifulSoup(doc.text, "html.parser")
        links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('http')]

        for link in links:
            cleaned_text += "\n\n" + fetch_content_url(link, crawled_urls)

        return cleaned_text

    return ""

In [49]:
# Save content to PDF
def save_content_to_pdf(content, filename):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    for line in content.split('\n'):
        pdf.multi_cell(0, 10, line)

    temp_dir = tempfile.gettempdir()
    temp_file_path = os.path.join(temp_dir, filename)
    pdf.output(temp_file_path)
    return temp_file_path

In [50]:




def parse_pdf(file_path):
    with open(file_path, 'rb') as f:
        buffer = BytesIO(f.read())
    reader = PdfReader(buffer)
    text = ''.join([page.extract_text() for page in reader.pages])
    os.remove(file_path)
    return text


In [51]:
def parse_docx(file_path):
    doc = Document(file_path)
    text = '\n'.join([para.text for para in doc.paragraphs])
    os.remove(file_path)
    return text


In [52]:
def parse_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    os.remove(file_path)
    return text

In [53]:


def get_azure_embeddings(texts):
    embeddings = []
    for text in texts:
        try:
            response = openai.Embedding.create(
                input=text,
                engine="text-embedding-ada-002"
            )
            embedding = response['data'][0]['embedding']
            embeddings.append(embedding)
        except Exception as e:
            logger.error(f"Error generating embedding for text: {text[:30]}... Error: {e}")
    return embeddings

In [70]:
def split_and_store_pdf(parsed_pdf, file_path):
    logger.info(f"Splitting and storing PDF file {file_path} content...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_text(parsed_pdf)

   
    chunk_list = list()
    for i,chunk in enumerate(chunks):
        embedding = get_azure_embeddings([chunk])[0]
        keywords = extract_keywords(chunk)
        data_properties = {
            "content": chunk,
            "keyword": keywords,
       
            
        }
        data_object = wvc.data.DataObject(properties=data_properties)
        chunk_list.append(data_object)
    chunk_client.data.insert_many(chunk_list)

In [72]:
file_path = "Paper.pdf"
parsed_pdf = parse_pdf(file_path)
split_and_store_pdf(parsed_pdf, file_path)

INFO:__main__:Splitting and storing PDF file Paper.pdf content...


WeaviateInsertManyAllFailedError: Every object failed during insertion. Here is the set of all errors: invalid text property 'keyword' on class 'Documents': not a string, but []interface {}

In [73]:
from typing import List


def download_and_chunk(src_url: str, chunk_size: int, overlap_size: int) -> List[str]:
    import requests
    import re

    response = requests.get(src_url)  # Retrieve source text
    source_text = re.sub(r"\s+", " ", response.text)  # Remove multiple whitespaces
    text_words = re.split(r"\s", source_text)  # Split text by single whitespace

    chunks = []
    for i in range(0, len(text_words), chunk_size):  # Iterate through & chunk data
        chunk = " ".join(text_words[max(i - overlap_size, 0): i + chunk_size])  # Join a set of words into a string
        chunks.append(chunk)
    return chunks


pro_git_chapter_url = "https://raw.githubusercontent.com/progit/progit2/main/book/01-introduction/sections/what-is-git.asc"
chunked_text = download_and_chunk(pro_git_chapter_url, 150, 25)

In [74]:
import weaviate.classes as wvc


collection_name = "GitBookChunk"

if client.collections.exists(collection_name):  # In case we've created this collection before
    client.collections.delete(collection_name)  # THIS WILL DELETE ALL DATA IN THE COLLECTION

chunks = client.collections.create(
    name=collection_name,
    properties=[
        wvc.config.Property(
            name="chunk",
            data_type=wvc.config.DataType.TEXT
        ),
        wvc.config.Property(
            name="chapter_title",
            data_type=wvc.config.DataType.TEXT
        ),
        wvc.config.Property(
            name="chunk_index",
            data_type=wvc.config.DataType.INT
        ),
    ],
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),  # Use `text2vec-openai` as the vectorizer
    generative_config=wvc.config.Configure.Generative.openai(),  # Use `generative-openai` with default parameters
)

INFO:httpx:HTTP Request: GET https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/schema/GitBookChunk "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST https://mshwf2zhqzoe7ypcmmnfvg.c0.us-east1.gcp.weaviate.cloud/v1/schema "HTTP/1.1 200 OK"


In [78]:
chunks_list = list()
for i, chunk in enumerate(chunked_text):
    data_properties = {
        "chapter_title": "What is Git",
        "chunk": chunk,
        "chunk_index": i
    }
    data_object = wvc.data.DataObject(properties=data_properties)
    chunks_list.append(data_object)
chunks.data.insert_many(chunks_list)

WeaviateInsertManyAllFailedError: Every object failed during insertion. Here is the set of all errors: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY